In [2]:
from SuperCNN import CNN

from SuperCNN import Parametros

from Auxiliares import takeinputs


In [3]:
val, pix, qcmval, qcmpix, pixelsconv, qcmpixconv = takeinputs()

convlay = [(1, "input"), (10, "relu")]

lay = [(64, "sigmoid"), (10, "softmax")]

parametros = Parametros(pix=pix, vales=val, qcmpix=qcmpix, qcmval=qcmval, infolay=lay, infoconvlay=convlay, padding=0, convrapide=True)

In [4]:
g = CNN(parametros)

g.train()


Percentage: 0.0
Percentage: 16.666666666666668
Percentage: 33.333333333333336
Percentage: 50.0
Percentage: 66.66666666666667
Percentage: 83.33333333333333


In [5]:
print(g.tauxlent())

89.91


In [ ]:
import pickle

with open('parameters.pickle', 'wb') as file:
    pickle.dump(g.parameters, file)
